In [ ]:
#!pip install ipywidgets matplotlib requests
#!jupyter nbextension enable --py widgetsnbextension

In [ ]:
#!pip install jupyterquiz

In [ ]:
# Виконати в окремій клітинці:
#!pip install ipywidgets matplotlib requests
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager  # Для JupyterLab

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
main_output = widgets.Output()

def show_question(question_index, html_content, answer_buttons):
    with main_output:
        clear_output(wait=True)
        display(
            widgets.VBox([
                widgets.HTML(f"<div style='font-size:18px; color:#2e3440; margin-bottom:12px'>{html_content}</div>"),
                widgets.VBox
                [header, radio, next_btn],
                layout==widgets.Layout(
                align_items='center',  # Центрування всіх дочірніх елементів
                width='100%'
        )
            ])
        )


In [ ]:
from collections import defaultdict
import numpy as np
def calculate_scores(user_answers, questions):
    """
    Підраховує бали по категоріях на основі вибраних відповідей.
    """
    scores = defaultdict(int)
    for q_idx, answer_idx in enumerate(user_answers):
        try:
            weights = questions[q_idx]["answers"][answer_idx]["weights"]
            for cat, points in weights.items():
                scores[cat] += points
        except (IndexError, KeyError) as e:
            print(f"Помилка у питанні {q_idx}: {str(e)}")
            continue
    return dict(scores)



def determine_archetype(scores):
    """
    Визначає архетип на основі балів по категоріях.
    """
    # Максимально можливі бали для кожної категорії (21 питання × 3 бали)
    max_possible = {"EI": 63, "KB": 63, "RM": 63, "AD": 63, "AS": 63}
    normalized = {k: (v / max_possible[k]) * 100 for k, v in scores.items()}

    # Матриця архетипів (коефіцієнти у порядку: EI, KB, RM, AD, AS)
    archetype_matrix = np.array([
        [-2.0, -2.0, -2.0,  2.0,  2.0],   # Маржинальний жнець
        [ 1.0,  1.0,  3.0,  2.0,  0.0],   # Рептилія
        [ 1.0,  2.0,  3.0,  3.0,  0.0],   # Арбітражний стабілізатор
        [-2.0, -2.0,  0.0,  3.0,  2.0],   # Провокатор хаосу
        [ 2.0,  2.0,  2.0,  3.0,  0.0],   # Емерджентний аналітик
        [ 2.0,  0.0,  3.0,  3.0,  2.0],   # Антифрагільний агент
        [ 2.0,  1.0,  0.0,  2.0,  3.0],   # Трендсеттер
        [ 3.0, -2.0,  0.0,  2.0,  2.0],   # Медіатор Духів
    ])

    archetypes = [
        "Маржинальний жнець",
        "Рептилія",
        "Арбітражний стабілізатор",
        "Провокатор хаосу",
        "Емерджентний аналітик",
        "Антифрагільний агент",
        "Трендсеттер",
        "Медіатор Духів"
    ]

    descriptions = [
        "Агресія, імпульсивність, ігнорування ризиків",
        "Системність, ригідність, дотримання правил",
        "Раціональність, пошук дисбалансів, арбітраж",
        "Любить волатильність, провокує ринок",
        "Адаптивний, технологічний, рефлексивний",
        "Виграє від стресу, асиметрична адаптація",
        "Впливає на думки, формує наративи",
        "Висока емпатія, інтуїція, емоційна мудрість"
    ]

    user_vector = np.array([
        normalized.get("EI", 0),
        normalized.get("KB", 0),
        normalized.get("RM", 0),
        normalized.get("AD", 0),
        normalized.get("AS", 0)
    ])

    similarities = archetype_matrix @ user_vector.T
    best_idx = int(np.argmax(similarities))
    similarity_percent = min(100, max(0, (similarities[best_idx] / 300) * 100))  # 300 — умовний максимум

    return archetypes[best_idx], round(similarity_percent, 1), descriptions[best_idx]

def get_category_percentages(scores, questions):
    """
    Конвертує абсолютні бали у відсотки для візуалізації.
    """
    max_scores = defaultdict(int)
    for q in questions:
        for answer in q["answers"]:
            for cat, points in answer["weights"].items():
                max_scores[cat] = max(max_scores[cat], points * len(q["answers"]))
    return {cat: min(100, (score / max_scores[cat]) * 100) if max_scores[cat] > 0 else 0
            for cat, score in scores.items()}



In [ ]:
from IPython.display import display, HTML

display(HTML("""
<style>
:root {
    --bg-color: #181a1b;
    --container-bg: #23272b;
    --text-color: #f1f1f1;
    --primary-accent: #4f8cff;
    --hover-accent: #357ae8;
    --border-radius: 22px;
}

/* Загальні стилі */
body, .jp-Notebook, .notebook_app {
    background: var(--bg-color) !important;
    color: var(--text-color);
    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif;
}

/* Контейнер питання */
.question-container {
    background: var(--container-bg);
    border-radius: var(--border-radius);
    padding: 40px 38px 28px;
    margin: 40px auto;
    max-width: 800px;
    font-size: 1.25rem;
    box-shadow: 0 8px 32px rgba(0,0,0,0.18);
    position: relative;
}

.question-title {
    color: var(--text-color);
    font-size: 1.35rem;
    font-weight: 700;
    margin-bottom: 24px;
    line-height: 1.4;
}

/* Стилі для вибору відповідей */
.widget-radio-box {
    background: transparent !important;
    gap: 12px;
}

.widget-radio {
    background: var(--container-bg);
    border: 2px solid #393e46;
    border-radius: 14px;
    padding: 16px 24px;
    margin: 0 0 15px;
    transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
}

.widget-radio:hover {
    border-color: var(--primary-accent);
    transform: translateY(-2px);
}

.widget-radio input[type="radio"]:checked + label {
    color: var(--primary-accent) !important;
}

.widget-radio label {
    font-size: 1.1rem !important;
    padding-left: 12px !important;
    color: #f1f1f1 !important;
    font-weight: 400 !important;
}

/* Кнопка "Далі" */
.widget-button {
    background: var(--primary-accent) !important;
    border-radius: 10px !important;
    padding: 12px 28px !important;
    font-size: 1.1rem !important;
    margin-top: 18px !important;
    transition: transform 0.2s, box-shadow 0.2s !important;
}

.widget-button:hover {
    background: var(--hover-accent) !important;
    transform: scale(1.02);
    box-shadow: 0 4px 16px rgba(79,140,255,0.28) !important;
}

/* Прогрес бар */
.progress-container {
    width: 90%;
    height: 14px;
    background: #2b2f33;
    border-radius: 8px;
    margin: 0 auto 18px;
    overflow: hidden;
}

.progress-fill {
    height: 100%;
    background: linear-gradient(90deg, var(--primary-accent) 0%, var(--hover-accent) 100%);
    transition: width 0.5s ease-in-out;
}

/* Адаптація для мобільних */
@media (max-width: 600px) {
    .question-container {
        padding: 28px 24px;
        margin: 20px 15px;
        border-radius: 18px;
    }
    
    .widget-radio {
        padding: 12px 18px;
    }
    
    .widget-button {
        width: 100%;
    }
}

/* Анімації */
@keyframes fadeIn {
    from { opacity: 0; transform: translateY(10px); }
    to { opacity: 1; transform: translateY(0); }
}

.widget-radio input[type="radio"]:checked + label {
    color: #4f8cff !important;
    font-weight: 600 !important;
} 
.question-container {
    animation: fadeIn 0.4s ease-out;
}
</style>
"""))


In [ ]:

# Твій список питань (вставити сюди всі 21 питання)
questions = [
    {
        "question": "Ваш портфель втратив 40% вартості за тиждень через макроекономічний шок. Ваші перші дії?",
        "answers": [
            {
                "text": "Продаю активніше на ключових рівнях, щоб активувати каскад ліквідацій інших трейдерів.",
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": "Маржинальний жнець",
                "explanation": "Агресія, loss aversion."
            },
            {
                "text": "Виконую заздалегідь прописані правила (напр., стоп-лоси) без додаткового аналізу.",
                "weights": {"EI": 3, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": "Рептилія",
                "explanation": "Емоційний контроль, планування."
            },
            {
                "text": "Шукаю цінові розбіжності між біржами для арбітражних угод.",
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": "Арбітражний стабілізатор",
                "explanation": "Раціональний підхід, але без повної адаптації."
            },
            {
                "text": "Докуповую активи з найвищою волатильністю, щоб посилити ринковий хаос.",
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": "Провокатор хаосу",
                "explanation": "Дуже ризиковано, може погіршити ситуацію."
            },
            {
                "text": "Аналізую, які сценарії не врахувала моя модель, і перебудовую стратегію.",
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": "Емерджентний аналітик",
                "explanation": "Адаптивність і аналітичність."
            },
            {
                "text": "Перерозподіляю капітал у активи, що виграють від кризи (золото, стейблкоїни).",
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": "Антифрагільний агент",
                "explanation": "Збалансований підхід до ризику."
            },
            {
                "text": "Готую серію постів/відео з прогнозом швидкого відновлення ринку.",
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": "Трендсеттер",
                "explanation": "Не впливає на реальний ризик."
            },
            {
                "text": "Роблю перерву, щоб 'відчути' ринок, перш ніж діяти.",
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": "Медіатор Духів",
                "explanation": "Інтуїтивний контроль, але важливо не затягувати з рішенням."
             }
        ]
    },
    {

    "question": "Ваш партнер по команді пропонує “сіру” схему для швидкого збагачення, але з високим ризиком регуляторних санкцій. Ви?",
    "answers": [
        {
            "text": "Розробляю план уникнення відстеження операцій.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресія + анонімність."
        },
        {
            "text": "Відмовляюсь і вимагаю припинити співпрацю.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 3},
            "archetype": "Рептилія",
            "explanation": "Ригідна відповідність правилам."
        },
        {
            "text": "Досліджую юридичні межі для мінімізації наслідків.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Аналіз лазівок."
        },
        {
            "text": "Пропоную ескалацію ризиків через підставні угоди.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Навмисне створення хаосу."
        },
        {
            "text": "Моделюю наслідки для репутації та фінансів команди.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Премортем-аналіз."
        },
        {
            "text": "Виношу рішення на команду, де кожен голосує.",
            "weights": {"EI": 0, "KB": 0, "RM": 1, "AD": 2, "AS": 2},
            "archetype": "Антифрагільний агент",
            "explanation": "Колективна відповідальність."
        },
        {
            "text": "Анонімно передаю інформацію про схему конкурентам.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Маніпуляція через інфлюенс."
        },
        {
            "text": "Відкладаю рішення, спостерігаючи за реакцією ринку.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 1, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивне відстрочення."
        }
        ]
    },
    {
    "question": "BTC падає з $95k до $78k за 20 хвилин. Ваш портфель -30%. Серце калатає, руки трусяться. Ваші дії?",
    "answers": [
        {
            "text": "Відкриваю шорт, щоб прискорити падіння та активувати ліквідації.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивний вплив на ринок."
        },
        {
            "text": "Чекаю, дотримуючись заздалегідь встановлених правил.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна системність."
        },
        {
            "text": "Купую на біржах з відстаючими цінами для арбітражу.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Експлуатація цінових розбіжностей."
        },
        {
            "text": "Провокую паніку, розміщуючи великі ордери на продаж.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Навмисне створення волатильності."
        },
        {
            "text": "Аналізую дані за допомогою AI для перевірки гіпотез.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічний підхід."
        },
        {
            "text": "Збільшую позиції, використовуючи падіння як можливість.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Асиметрична ставка."
        },
        {
            "text": "Публікую оптимістичні прогнози, щоб змінити ринковий наратив.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Маніпуляція інформаційним полем."
        },
        {
            "text": "Роблю паузу, щоб відновити емоційний баланс перед діями.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивний контроль."
        }
 ]
    },
    {
    "question": "З'являється принципово нова технологія торгівлі (квантові алгоритми), яка дає +50% precision. Але коштує $50k і потребує 6 місяців навчання. Ваша реакція:",
    "answers": [
        {
            "text": "Інвестую всі доступні кошти, щоб першим отримати перевагу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивне захоплення інновацій."
        },
        {
            "text": "Чекаю, поки технологія пройде тестування на ринку.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Консервативний підхід."
        },
        {
            "text": "Створюю консорціум з іншими трейдерами для спільного фінансування.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 1},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Колективна оптимізація."
        },
        {
            "text": "Експериментую з дешевшими аналогами, щоб спровокувати хаус у ринкових паттернах.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Дестабілізація через імітацію."
        },
        {
            "text": "Розробляю власний алгоритм на основі відкритих джерел.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна самостійність."
        },
        {
            "text": "Використовую паніку конкурентів для арбітражу на їхніх помилках.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Асиметрична адаптація."
        },
        {
            "text": "Формую спільноту ранніх користувачів для масштабування впливу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Створення інформаційного поля."
        },
        {
            "text": "Аналізую власну готовність через медитацію/психотехніки.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивна оцінка ризиків."
        }
 ]
    },
    {
    "question": "У чаті новачок плаче, що втратив останні $500. Ваша перша внутрішня реакція:",
    "answers": [
        {
            "text": "Думаю, що це його помилка через відсутність контролю ризиків.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Жорстка раціональність, низька емпатія."
        },
        {
            "text": "Відчуваю співчуття, згадуючи власні втрати на старті.",
            "weights": {"EI": 3, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Емоційна пам’ять, базова емпатія."
        },
        {
            "text": "Аналізую його дії, щоб не повторити таких помилок у майбутньому.",
            "weights": {"EI": 0, "KB": 2, "RM": 0, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Навчання на чужих кейсах."
        },
        {
            "text": "Використовую ситуацію, щоб запропонувати йому платний продукт чи консультацію.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Маніпулятивний підхід."
        },
        {
            "text": "Шукаю способи допомогти йому конкретними порадами чи ресурсами.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 1},
            "archetype": "Емерджентний аналітик",
            "explanation": "Конструктивна підтримка."
        },
        {
            "text": "Ігнорую ситуацію, вважаючи, що кожен сам несе відповідальність.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Індивідуалізм, низька залученість."
        },
        {
            "text": "Відчуваю провину за свої успіхи на тлі його втрати.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 1},
            "archetype": "Трендсеттер",
            "explanation": "Соціальна рефлексія, підвищена емпатія."
        },
        {
            "text": "Відчуваю емоційний біль на фізичному рівні, як ніби це сталося зі мною.",
            "weights": {"EI": 3, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Гіперемпатія, тілесна реакція."
        }
 ]
    },
    {
    "question": "Під час відеодзвінка ви помічаете, що один колега виглядає напружено, але нічого не каже. Ви:",
    "answers": [
        {
            "text": "Ігнорую, зосереджуючись на власних задачах.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Фокус на результатах, низька емпатія."
        },
        {
            "text": "Чітко запитую: 'Чи потрібна допомога?'",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Рептилія",
            "explanation": "Прямолінійність, дотримання протоколу."
        },
        {
            "text": "Аналізую його останні угоди, щоб знайти причину напруги.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Раціональний аналіз."
        },
        {
            "text": "Навмисно створюю конфліктну ситуацію, щоб викликати його реакцію.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Дестабілізація для отримання інформації."
        },
        {
            "text": "Пропоную технічні інструменти для моніторингу стресу (напр., біодатчики).",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічне рішення."
        },
        {
            "text": "Змінюю тему, щоб зменшити тиск на нього.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 2},
            "archetype": "Антифрагільний агент",
            "explanation": "Непряма підтримка через зменшення стресу."
        },
        {
            "text": "Публічно підкреслюю його внесок, щоб підвищити впевненість.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Маніпуляція соціальним статусом."
        },
        {
            "text": "Після дзвінка надсилаю повідомлення з пропозицією поговорити.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 1},
            "archetype": "Медіатор Духів",
            "explanation": "Делікатна непряма комунікація."
           }
 ]
    },
    {
    "question": "Ваша прибуткова стратегія експлуатує помилки retail-трейдерів. Заробляєте, але завдаєте шкоди 'простим людям':",
    "answers": [
        {
            "text": "Продовжую, використовуючи їхні помилки для максимізації прибутку.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивна експлуатація слабкостей."
        },
        {
            "text": "Повністю відмовляюсь від стратегії через порушення власних принципів.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 3},
            "archetype": "Рептилія",
            "explanation": "Ригідна відповідність внутрішнім правилам."
        },
        {
            "text": "Переношу фокус на великі інституційні угоди, уникаючи retail-ринку.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Системне уникнення етичних конфліктів."
        },
        {
            "text": "Навмисно підсилюю хаос, щоб retail-трейдери швидше виходили з ринку.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Дестабілізація для власної вигоди."
        },
        {
            "text": "Модифікую стратегію з використанням AI для зменшення негативного впливу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічне вирішення етичних проблем."
        },
        {
            "text": "Створюю безкоштовні курси з ризик-менеджменту для новачків.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Антифрагільний агент",
            "explanation": "Підвищення стійкості ринку через освіту."
        },
        {
            "text": "Публікую детальні аналізи ризиків стратегії у соцмережах.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Формування прозорого інформаційного поля."
        },
        {
            "text": "Вводжу 'етичний податок' — частину прибутку віддаю на фінансову грамотність.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 2},
            "archetype": "Медіатор Духів",
            "explanation": "Баланс між прибутком і соціальною відповідальністю."
        }
 ]
    },
     {
    "question": "Ваш ШІ-помічник дає сигнал BUY на NVIDIA/SOL, але ваш досвід каже SELL. Конфлікт триває 30 хвилин. Ви:",
    "answers": [
        {
            "text": "Виконую сигнал ШІ, довіряючи його обсягу даних.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Максимізація ефективності через технології."
        },
        {
            "text": "Ігнорую сигнал ШІ, дію згідно з власним досвідом.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідність, самостійність."
        },
        {
            "text": "Розділяю позицію навпіл, щоб мінімізувати ризик.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Балансування протилежних стратегій."
        },
        {
            "text": "Шукаю додаткові думки серед професіоналів для прийняття рішення.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Соціальна ескалація невизначеності."
        },
        {
            "text": "Аналізую, чому ШІ пропонує протилежне, і коригую свою модель.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Пошук причин розбіжності, навчання."
        },
        {
            "text": "Використовую ситуацію для короткострокового арбітражу між сигналами.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Експлуатація конфлікту для вигоди."
        },
        {
            "text": "Публікую дилему у соцмережах, щоб вплинути на громадську думку.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Інформаційний вплив."
        },
        {
            "text": "Роблю паузу, концентруюсь на внутрішніх відчуттях, перш ніж діяти.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивна інтеграція рішень."
      }
 ]
    },
   { 
    "question": "Команда розколота: 50% хочуть all-in у AI-акції, 50% — в gold як hedge. Дедлайн — година:",
    "answers": [
        {
            "text": "Активно підтримую ризикований варіант (AI), щоб прискорити рішення.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Провокація конфлікту для отримання переваги."
        },
        {
            "text": "Наполягаю на консервативному підході (gold), посилаючись на історичні дані.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна прив’язка до перевірених стратегій."
        },
        {
            "text": "Пропоную розподіл 50/50 для збереження командної стабільності.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Баланс інтересів."
        },
        {
            "text": "Запускаю дискусію про третій варіант (напр., криптовалютні стейблкоїни), щоб дестабілізувати процес.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Саботаж консенсусу."
        },
        {
            "text": "Демонструю моделі ризиків/прибутковості обох стратегій з використанням AI.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Далекоглядний аналіз."
        },
        {
            "text": "Виходжу з колективного рішення, реалізуючи власну стратегію.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Автономія в умовах хаосу."
        },
        {
            "text": "Організовую дебати з обов’язковим фіксуванням аргументів у спільному документі.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Структурована комунікація."
        },
        {
            "text": "Пропоную команді 10-хвилинну паузу для відновлення емоційного балансу перед голосуванням.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Непряме керування через емоційний інтелект."
    }
 ]
    },
   {
    "question": "Партнер пропонує стратегію, яка вам не подобається. Як відреагуєте?",
    "answers": [
        {
            "text": "Різко критикую ідею, називаючи її безперспективною.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Пряма агресія для утвердження домінування."
        },
        {
            "text": "Чітко аргументую свої сумніви, посилаючись на дані.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 3},
            "archetype": "Рептилія",
            "explanation": "Структурована асертивність."
        },
        {
            "text": "Погоджуюсь публічно, але таємно саботую реалізацію.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Пасивно-агресивна дестабілізація."
        },
        {
            "text": "Посилаюсь на авторитет експертів, щоб дискредитувати ідею.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Маніпуляція через об’єктивні джерела."
        },
        {
            "text": "Запускаю AI-модель для порівняння стратегій з історичними даними.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна об’єктивність."
        },
        {
            "text": "Делегую йому повну відповідальність за результат.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Уникнення особистих ризиків."
        },
        {
            "text": "Організовую відкрите голосування, щоб зняти з себе відповідальність.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Соціальна маніпуляція."
        },
        {
            "text": "Пропоную спільну медитацію для пошуку консенсусу.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 1},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивна інтеграція конфлікту."
        }
 ]
    },
   {
    "question": "За 30 секунд потрібно вибрати: LONG BTC, SHORT ETH, арбітраж BTC/ETH, або залишитись у cash. Одночасно у чаті 50 трейдерів обговорюють різні сигнали. Ви перевантажені інформацією:",
    "answers": [
        {
            "text": "Вкладаю максимально в BTC, ігноруючи інші опції.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивний фокус на високому потенціалі."
        },
        {
            "text": "Обираю гарантований арбітраж, уникаючи стресу.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Раціональне уникнення конфлікту."
        },
        {
            "text": "Залишаюсь у cash через параліч аналізу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Дисфункція при порушенні системності."
        },
        {
            "text": "Розподіляю капітал між усіма опціями.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Ескалація невизначеності."
        },
        {
            "text": "Блокуваю чат, концентруючись на власних алгоритмах.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Фільтрація шуму через технології."
        },
        {
            "text": "Використовую випадковий вибір (напр., підкидання монети).",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Стратегічна рандомізація."
        },
        {
            "text": "Проводжу дихальні вправи для відновлення фокусу.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Mindfulness для зниження активності ACC."
        },
        {
            "text": "Збільшую розмір усіх позицій, щоб компенсувати хаос.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Ейфорійна ескалація."
        }
 ]
    },
   { 
    "question": "Після 5 років успішного трейдингу ви відчуваєте втому. Гроші є, але немає мотивації. Друзі кажуть 'продовжуй', родина 'відпочивай'. Що робите?",
    "answers": [
        {
            "text": "Різко збільшую розмір угод у екзотичних активах для відчуття азарту.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Пошук нових стимулів через екстремальний ризик."
        },
        {
            "text": "Повністю припиняю торгівлю на рік, зосередившись на подорожах.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Радикальний 'рестарт' системи."
        },
        {
            "text": "Переходжу на алгоритмічну торгівлю з обмеженим щоденним лімітом угод.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Системна мінімізація навантаження."
        },
        {
            "text": "Створюю провокативний контент про ринкові маніпуляції для підживлення інтересу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Генерація зовнішнього хаосу як джерела енергії."
        },
        {
            "text": "Розробляю власний ШІ-трейдер із функцією автономного навчання.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна трансформація досвіду."
        },
        {
            "text": "Переношу капітал у венчурні проекти, де ризик компенсується соціальним впливом.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Асиметричний перехід до нової парадигми."
        },
        {
            "text": "Запускаю YouTube-канал з кейсами 'як я розорив 100 новачків'.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Маніпуляція соціальним статусом для підтримки его."
        },
        {
            "text": "Практикую 40-денне мовчання в монастирі для переосмислення цінностей.",
            "weights": {"EI": 3, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Глибока інтроспекція через радикальний детокс."
       }
 ]
    },
   { 
    "question": "Як емоції впливають на ваші торгові рішення?",
    "answers": [
        {
            "text": "Практикую техніки контролю (напр., дихання) для повного усунення емоцій.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Демонстрація надмірної самоконтролю."
        },
        {
            "text": "Рідко дозволяю емоціям впливати — лише у 1-2 угодах на місяць.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Системний підхід з винятками."
        },
        {
            "text": "Емоції час від часу переважують (напр., під час флеш-крешів).",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Хаотичні спалахи."
        },
        {
            "text": "Використовую емоційні сигнали як частину стратегії (напр., FOMO).",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Інтеграція емоцій у наратив."
        },
        {
            "text": "Аналізую кожен емоційний вплив постфактум через журнал угод.",
            "weights": {"EI": 3, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Рефлексивний дебіазинг."
        },
        {
            "text": "Емоції — ключовий фактор у більшості моїх рішень.",
            "weights": {"EI": 3, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Гіперемпатична модель."
        },
        {
            "text": "Відокремлюю емоції за допомогою алгоритмів (напр., автоматичні стоп-лоси).",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Технологічне виключення."
        },
        {
            "text": "Не аналізую це — головне, що стратегія приносить прибуток.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Прагматичне ігнорування."
      }
 ]
    },
   {
    "question": "Ваш улюблений аналітичний метод не працює вже місяць. Що робите?",
    "answers": [
        {
            "text": "Збільшую кількість угод за цим методом, сподіваючись на відновлення результату.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Ескалація зусиль, ризикована впертість."
        },
        {
            "text": "Повністю відмовляюсь і переходжу на альтернативні підходи.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна зміна стратегії при невдачі."
        },
        {
            "text": "Комбіную цей метод з іншими для пошуку синергії.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Диверсифікація аналітики."
        },
        {
            "text": "Використовую метод для протилежних сигналів (контртренд).",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Інверсія стратегії."
        },
        {
            "text": "Тестую метод на нових ринках або інструментах.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Експериментування з новими даними."
        },
        {
            "text": "Обговорюю проблему в професійній спільноті, шукаю колективний досвід.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Соціальна адаптація."
        },
        {
            "text": "Питаю думку у визнаних експертів галузі.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Орієнтація на авторитети."
        },
        {
            "text": "Довіряю інтуїції, вирішуючи, чи варто продовжувати.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивне прийняття рішень."
        }
 ]
    },
   {
    "question": "Ваш торговий бот заробив +300% за рік, але сьогодні зробив серію дивних угод, які ви до кінця не розумієте. Портфель у -15% за день. Боти 'не помиляються', але щось не так. Ви:",
    "answers": [
        {
            "text": "Вимикаю бота негайно, переходжу на ручне управління.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресія через втрату контролю."
        },
        {
            "text": "Залишаю все як є, вважаючи це статистичним шумом.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна відповідність системі."
        },
        {
            "text": "Модифікую алгоритм з урахуванням нових ринкових патернів.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Раціональна оптимізація."
        },
        {
            "text": "Створюю другого бота з протилежною стратегією для балансу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Генерація контртренду."
        },
        {
            "text": "Запускаю ML-аналіз для виявлення аномалій у роботі бота.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна діагностика."
        },
        {
            "text": "Звертаюся до форумів алготрейдерів для колективного аналізу.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Соціальна адаптація."
        },
        {
            "text": "Консультуюся з розробниками для перевірки коду.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Орієнтація на експертну думку."
        },
        {
            "text": "Прислухаюся до інтуїції, тимчасово обмежуючи роботу бота.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивний контроль."
         }
 ]
    },
   {
    "question": "Світова регуляторна криза: Китай забороняє трейдинг, США вводять 60% податок на крипту, ЄС запускає CBDC. Яка ваша стратегія?",
    "answers": [
        {
            "text": "Швидко переїжджаю до країни з мінімальними обмеженнями для трейдерів.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивна мобільність заради переваги."
        },
        {
            "text": "Оперативно адаптую свою стратегію до нових регуляторних вимог.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Системна гнучкість у межах правил."
        },
        {
            "text": "Повністю переходжу на DeFi та децентралізовані біржі.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 3, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Ухилення через інфраструктурну зміну."
        },
        {
            "text": "Використовую різницю між юрисдикціями для арбітражу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Експлуатація глобального дисбалансу."
        },
        {
            "text": "Використовую ШІ для автоматичної оптимізації під нові правила.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна адаптація."
        },
        {
            "text": "Об'єднуюсь із трейдерами для створення міжнародної захисної спільноти.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Колективна стійкість."
        },
        {
            "text": "Лобіюю інтереси через професійні організації та медіа.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Вплив на регуляторне поле через комунікацію."
        },
        {
            "text": "Роблю паузу, спостерігаю за розвитком подій, щоб не поспішати з рішенням.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Стратегічне очікування та інтроспекція."
         }
 ]
    },
   {
    "question": "Ваш портфель у мінусі 60%. Ви отримали пропозицію від інвестора: 'Я дам вам $10k, якщо ви ризикнете всім і зробите x2 за тиждень, або борг'. Ваші дії:",
    "answers": [
        {
            "text": "Ризикую всіма коштами на найбільш волатильний актив.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивний азарт."
        },
        {
            "text": "Відмовляюсь, дотримуючись власних правил ризик-менеджменту.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна дисципліна."
        },
        {
            "text": "Використовую арбітраж між біржами для гарантованого прибутку.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Системне уникнення ризику."
        },
        {
            "text": "Публікую опитування в соцмережах, щоб інші вирішили за мене.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Екстерналізація відповідальності."
        },
        {
            "text": "Аналізую тисячі сценаріїв у ML-моделі для оптимального рішення.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічний підхід."
        },
        {
            "text": "Пропоную інвестору розділити ризик порівну.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Колективне зменшення втрат."
        },
        {
            "text": "Шукаю лазівки в умовах угоди для мінімізації збитків.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Маніпуляція правилами."
        },
        {
            "text": "Роблю паузу, щоб відчути, чи варто приймати пропозицію.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивна оцінка."
         }
 ]
    },
   {
    "question": "Індекс страху VIX = 85. Всі активи корелюють -90%. Готівки немає. Ваші дії?",
    "answers": [
        {
            "text": "Беру максимальний кредит, щоб викупити 'дно'.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивна гра на відновлення."
        },
        {
            "text": "Продаю всі активи, переводячи капітал у стейблкоїни.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна стратегія збереження."
        },
        {
            "text": "Шукаю арбітражні можливості між некорельованими активами.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Раціональний пошук лазівок."
        },
        {
            "text": "Запускаю платний курс 'Як заробити на кризі'.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Маніпуляція панікою."
        },
        {
            "text": "Використовую AI для аналізу історичних даних подібних криз.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна адаптація."
        },
        {
            "text": "Координую свої дії з міжнародною спільнотою трейдерів.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Колективна стійкість."
        },
        {
            "text": "Переводжу активи в золото та облігації.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
            "archetype": "Трендсеттер",
            "explanation": "Консервативна адаптація."
        },
        {
            "text": "Відключаю термінал і зосереджуюсь на духовних практиках.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивне відсторонення."
   }
 ]
    },
   {
    "question": "SEC анонсували 'нові правила' для DeFi за місяць. Деталей немає. 40% портфеля в DeFi:",
    "answers": [
        {
            "text": "Негайно продаю всі De-активи, уникаючи ризику регуляторних санкцій.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 2, "AS": 3},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивне уникнення збитків через регуляторні ризики"
        },
        {
            "text": "Залишаю портфель без змін, доки не з’являться офіційні деталі.",
            "weights": {"EI": 2, "KB": 1, "RM": 3, "AD": 0, "AS": 2},
            "archetype": "Рептилія",
            "explanation": "Ригідна дисципліна та контроль ризиків"
        },
        {
            "text": "Хеджую ризики через традиційні фінансові інструменти (напр., ф’ючерси).",
            "weights": {"EI": 2, "KB": 1, "RM": 2, "AD": 3, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Системне зменшення експозиції через диверсифікацію"
        },
        {
            "text": "Створюю панічні продажі в соцмережах, щоб купити активи дешевше.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 2},
            "archetype": "Провокатор хаосу",
            "explanation": "Маніпуляція ринковою панікою для власної вигоди"
        },
        {
            "text": "Аналізую історичні кейси регуляторних змін за допомогою AI.",
            "weights": {"EI": 1, "KB": 2, "RM": 2, "AD": 3, "AS": 1},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічний прогноз та адаптивний підхід"
        },
        {
            "text": "Збільшую частку DeFi, вважаючи регулювання знаком майбутнього масового прийняття.",
            "weights": {"EI": 3, "KB": 1, "RM": 2, "AD": 3, "AS": 2},
            "archetype": "Антифрагільний агент",
            "explanation": "Асиметрична ставка на легалізацію технологій"
        },
        {
            "text": "Приєднуюсь до колективних лобістських зусиль галузі.",
            "weights": {"EI": 2, "KB": 1, "RM": 1, "AD": 2, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Активний вплив на регуляторний процес"
        },
        {
            "text": "Роблю паузу, сприймаючи невизначеність як частину ринкового циклу.",
            "weights": {"EI": 3, "KB": 0, "RM": 2, "AD": 2, "AS": 1},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивна адаптація до ринкових умов"
     }
 ]
    },
   {
    "question": "Чи завжди ви ретельно досліджуєте всі активи перед інвестуванням?",
    "answers": [
        {
            "text": "Завжди проводжу повний фундаментальний та технічний аналіз.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Рептилія/Арбітражний стабілізатор",
            "explanation": "Системність."
        },
        {
            "text": "Частково аналізую, частково довіряю внутрішнім інсайтам.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивний баланс."
        },
        {
            "text": "Використовую AI-інструменти для автоматизованої оцінки.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна ефективність."
        },
        {
            "text": "Довіряю думці експертів або соціальній думці.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Орієнтація на авторитети."
        },
        {
            "text": "Дію швидко, спираючись на тренди без глибокого аналізу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець/Провокатор хаосу",
            "explanation": "Імпульсивність."
        },
        {
            "text": "Аналізую лише великі угоди, дрібні — інтуїтивно.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Селективний підхід."
   }
 ]
    },
   {
    "question": "ШІ починає торгувати краще за 95% людей. Через 3 роки people-трейдери можуть стати непотрібними. Ваша стратегія виживання:",
    "answers": [
        {
            "text": "Спеціалізуюсь на ринках, де можна експлуатувати слабкості та прогнозованість торгових ШІ.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Пошук нових вразливостей для полювання."
        },
        {
            "text": "Повністю виходжу з трейдингу, оскільки моя перевірена система більше не працює.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна реакція на повну зміну парадигми."
        },
        {
            "text": "Переходжу в управління великим капіталом, де потрібен людський нагляд та комунікація з клієнтами.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Пошук нової стабільної, системної ролі."
        },
        {
            "text": "Зосереджуюсь на активах з максимальною непередбачуваністю, де ШІ дає збої.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Пошук ніші, де хаос залишається ключовим фактором."
        },
        {
            "text": "Створюю власний ШІ-фонд, розробляючи та вдосконалюючи власні пропрієтарні алгоритми.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Еволюція від користувача до творця технологій."
        },
        {
            "text": "Створюю гібридні команди, поєднуючи сильні сторони людей (інтуїція, стратегія) та ШІ (швидкість, дані).",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Використання стресу системи для створення нової, сильнішої структури."
        },
        {
            "text": "Очолюю рух 'людських трейдерів', створюючи освітній контент та спільноту для збереження 'людської переваги'.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Створення наративу та лідерство у новій ніші."
        },
        {
            "text": "Спеціалізуюсь на ESG-інвестуванні та інших сферах, де ключову роль відіграють людські цінності, а не чиста математика.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Перехід у нішу, що керується етикою та інтуїцією."
        }
        ]
    }
]

# Функції для підрахунку (які я вже надав)
def calculate_scores(user_answers, questions):
    scores = {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0}
    
    for i, answer_index in enumerate(user_answers):
        if i < len(questions) and answer_index < len(questions[i]["answers"]):
            weights = questions[i]["answers"][answer_index]["weights"]
            for category, points in weights.items():
                scores[category] += points
    
    return scores

def determine_archetype_advanced(scores):
    """
    Визначає архетип на основі таблиці з зображення
    """
    # Знаходимо топ-категорії та найнижчі
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    
    # Патерни архетипів з таблиці
    archetype_patterns = {
        "Маржинальний жнець": {
            "high": [],
            "low": ["KB", "RM", "EI"],
            "medium_high": ["AS", "AD"],
            "description": "Агресія, імпульсивність, ігнорування ризиків, маніпуляція, схильність до all-in, низька емпатія"
        },
        "Рептилія": {
            "high": ["RM", "AS"],
            "medium": ["EI", "KB"],
            "low": [],
            "description": "Системність, ригідність, дотримання правил, уникає змін, низька адаптивність, мінімізація втрат"
        },
        "Арбітражний стабілізатор": {
            "high": ["RM", "AD", "KB"],
            "medium": ["EI"],
            "low": [],
            "description": "Раціональність, пошук дисбалансів, диверсифікація, арбітраж, уникає емоційних рішень"
        },
        "Провокатор хаосу": {
            "high": ["AD"],
            "medium_high": ["AS"],
            "low": ["KB", "EI"],
            "description": "Любить волатильність, провокує ринок, використовує хаос, схильний до ризикованих стратегій"
        },
        "Емерджентний аналітик": {
            "high": ["AD", "KB", "RM", "EI"],
            "low": [],
            "description": "Адаптивний, технологічний, рефлексивний, аналітичний, швидко змінює стратегії, використовує AI/ML"
        },
        "Антифрагільний агент": {
            "high": ["AD", "RM", "EI", "AS"],
            "low": [],
            "description": "Виграє від стресу, любить асиметрію, використовує невизначеність як ресурс, гнучкий"
        },
        "Трендсеттер": {
            "high": ["AS", "EI", "AD"],
            "medium": ["KB"],
            "low": [],
            "description": "Впливає на думки інших, формує наративи, соціально активний, високий емоційний інтелект"
        },
        "Медіатор Духів": {
            "high": ["EI", "AS", "AD"],
            "low": ["KB"],
            "description": "Висока емпатія, інтуїція, емоційна мудрість, схильність до рефлексії, балансує емоції та раціо"
        }
    }
    

In [ ]:
main_output = widgets.Output()

In [ ]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt
import requests  # Для Telegram інтеграції

class RiskToleranceTest:
    def __init__(self, questions):
        self.questions = questions
        self.current_question = 0
        self.user_answers = []
        self.scores = {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0}
        self.output = widgets.Output()
        self._init_styles()
        
    def _init_styles(self):
        display(HTML("""
        <style>
            .progress-container { /* ... */ }
            .question-container {
                background: #23272b;
                color: #f1f1f1;
                border-radius: 22px;
                padding: 40px;
                margin: 20px auto;
                max-width: 800px;
                box-shadow: 0 8px 32px rgba(0,0,0,0.18);
            }
            .progress-bar-container {
                background: rgba(255,255,255,0.1);
                border-radius: 10px;
                height: 8px;
                margin: 15px 0;
            }
            .progress-fill {
                background: linear-gradient(90deg, #00d4aa, #00a8ff);
                height: 100%;
                border-radius: 10px;
                transition: width 0.5s ease;
            }
            .question-text {
                font-size: 1.3em;
                font-weight: 600;
                margin: 25px 0;
                line-height: 1.4;
            }
            .widget-button {
                background: #4f8cff !important;
                color: white !important;
                border-radius: 18px !important;
                transition: all 0.3s ease !important;
            }
            .widget-button:hover {
                background: #357ae8 !important;
                transform: scale(1.05);
            }
        </style>
        """))
        
    def start(self):
        display(self.output)
        self._show_question()

    def _show_question(self):
        with self.output:
            clear_output(wait=True)
        
        if self.current_question >= len(self.questions):
            self._show_results()
            return
        
        q = self.questions[self.current_question]
        progress = (self.current_question / len(self.questions)) * 100
        
        # Заголовок питання
        header = widgets.HTML(
            f"""
            <div class="question-container">
                <div class="progress-container">
                    <div class="progress-fill" style="width:{progress}%"></div>
                </div>
                <h2 class="question-title">Питання {self.current_question + 1} з {len(self.questions)}</h2>
                <div class="question-text">{q['question']}</div>
            </div>
            """
        )
        
        # Віджети для відповідей
        radio = widgets.RadioButtons(
            options=[ans["text"] for ans in q["answers"]],
            description='Ваша відповідь:',
            disabled=False
        )
        
        # Кнопка "Далі"
        next_btn = widgets.Button(
            description='Далі' if self.current_question < len(self.questions) - 1 else 'Завершити тест',
            layout=widgets.Layout(
                width='220px',
                height='48px',
                margin='30px auto 0 auto',
                border_radius='18px'
            ),
            style=widgets.ButtonStyle(
                button_color='#4f8cff'
            )
        )
        next_btn.on_click(self._handle_answer)  # Підписка на клік
        
        # Відображення інтерфейсу
        display(widgets.VBox([header, radio, next_btn])) 
        

    def _handle_answer(self, b):
        with self.output:
            clear_output(wait=True)
            selected_idx = radio.index
            
            if selected_idx is None:
                display(HTML("<div style='color:#ff6b6b; padding:10px;'>Оберіть відповідь!</div>"))
                self._show_question()
                return

            # Оновлення балів
            self.user_answers.append(selected_idx)
            for cat, val in self.questions[self.current_question]['answers'][selected_idx]['weights'].items():
                self.scores[cat] += val

            self.current_question += 1
            self._show_question()

    def _show_results(self):
        with self.output:
            clear_output(wait=True)
            
            # Розрахунок результатів
            archetype, similarity, description = determine_archetype(self.scores)
            
            # Візуалізація
            result_html = f"""
            <div class="question-container">
                <h2 style="color:#4f8cff; text-align:center; margin-bottom:30px;">📊 Ваші результати</h2>
                
                <div style="background:#181a1b; border-radius:12px; padding:25px; margin-bottom:25px;">
                    <h3 style="color:#4f8cff; margin-bottom:15px;">{archetype} ({similarity}%)</h3>
                    <p style="font-size:1.1em; line-height:1.6;">{description}</p>
                </div>
                
                {self._create_feedback_form()}
            </div>
            """
            
            display(HTML(result_html))
            self._send_to_telegram(archetype, similarity)

    def _create_feedback_form(self):
        feedback_html = """
        <div style="margin-top:40px;">
            <h4 style="color:#4f8cff; margin-bottom:20px;">💬 Зворотній зв'язок</h4>
            <textarea id="feedback" placeholder="Ваші коментарі..." 
                style="width:100%; height:100px; padding:10px; border-radius:8px; background:#181a1b; color:white; border:1px solid #4f8cff;"></textarea>
            <button onclick="submitFeedback()" 
                style="background:#4f8cff; color:white; border:none; padding:12px 24px; border-radius:8px; margin-top:15px; cursor:pointer;">
                Надіслати відгук
            </button>
        </div>
        
        <script>
            function submitFeedback() {
                const feedback = document.getElementById('feedback').value;
                google.colab.kernel.invokeFunction('notebook.save_feedback', [feedback], {});
                alert('Дякуємо за відгук!');
            }
        </script>
        """
        return feedback_html

    def _send_to_telegram(self, archetype, similarity):
        try:
            TELEGRAM_TOKEN = "8021521612:AAFHa0rQcPhgyBh7y3Q5ggsOkNCu_5RDvyo"  # Використовуйте змінні середовища
            CHAT_ID = "298306752"
            
            message = (
                f"Новий результат тесту!\n"
                f"Архетип: {archetype}\n"
                f"Схожість: {similarity}%\n"
                f"Бали: {self.scores}"
            )
            
            requests.post(
                f"https://api.telegram.org/bot8021521612:AAFHa0rQcPhgyBh7y3Q5ggsOkNCu_5RDvyo/sendMessage",
                json={"chat_id": 298306752, "text": message}
            )
        except Exception as e:
            print(f"Помилка відправки в Telegram: {str(e)}")

# Запуск тесту
test = RiskToleranceTest(questions)  # questions має бути визначено
test.start()


In [1]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

current_question = 0
user_answers = []
scores = {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0}

def show_question(question_index):
    clear_output()
    q = questions[question_index]
    progress = (question_index / len(questions)) * 100

    # HTML з прогрес-баром і питанням
    html_content = f"""
    <div style="background:#23272b; color:#f1f1f1; border-radius:22px; padding:40px 38px 28px 38px; margin:40px auto 28px auto; max-width:800px; font-size:1.25em; box-shadow: 0 8px 32px rgba(0,0,0,0.18);">
        <div style="margin-bottom:20px;">
            <div style="color:#4f8cff; font-size:1.15em; margin-bottom:10px;">
                <b>Питання {question_index + 1} з {len(questions)}</b>
            </div>
            <div style="background: rgba(255,255,255,0.1); border-radius: 10px; height: 8px; margin: 15px 0;">
                <div style="background: linear-gradient(90deg, #00d4aa, #00a8ff); height: 100%; border-radius: 10px; width:{progress}%;"></div>
            </div>
        </div>
        <div style="font-size:1.3em; font-weight:600; margin-bottom:25px; line-height:1.4;">
            {q['question']}
        </div>
    </div>
    """

         # RadioButtons для відповідей
    radio = widgets.RadioButtons(
        options=[ans["text"] for ans in q["answers"]],
        layout=widgets.Layout(width='100%'),
        style={'description_width': '0px', 'font_weight': 'bold', 'font_size': '1.1em', 'text_color': '#f1f1f1'}
    )

# Кнопка "Далі"
    next_btn = widgets.Button(
        description='Далі' if question_index < len(questions) - 1 else 'Завершити тест',
        layout=widgets.Layout(
            width='220px',
            height='48px',
            margin='30px auto 0 auto',
            border_radius='18px'
        ),
        style=widgets.ButtonStyle(
            button_color='#4f8cff'
        )
    )

# На початку ноутбука:
user_answers = []
current_question = 0

# ... далі у функції show_question:
radio = widgets.RadioButtons(
    options=[ans["text"] for ans in q["answers"]],
    layout=widgets.Layout(width='100%'),
    style={'description_width': '0px', 'font_weight': 'bold', 'font_size': '1.1em', 'text_color': '#f1f1f1'}
)

next_btn = widgets.Button(
    description='Далі' if question_index < len(questions) - 1 else 'Завершити тест',
    layout=widgets.Layout(
        width='220px',
        height='48px',
        margin='30px auto 0 auto',
        border_radius='18px'
    ),
    style=widgets.ButtonStyle(
        button_color='#4f8cff'
    )
)

    def on_next_click(b):
        global current_question
        if radio.index is None:
            display(HTML("<p style='color:#ff6b6b; margin:15px 0;'>Будь ласка, оберіть відповідь!</p>"))
            return
        user_answers.append(radio.index)
        current_question += 1
        if current_question < len(questions):
        show_question(current_question)
        else:
        show_results()

    next_btn.on_click(on_next_click)  # Правильний відступ
    
import requests

def send_feedback_to_telegram(feedback_text, archetype, similarity_percent):
    TELEGRAM_TOKEN = "8021521612:AAFHa0rQcPhgyBh7y3Q5ggsOkNCu_5RDvyo"  # Замініть на токен з BotFather
    CHAT_ID = "298306752"       # Замініть на ваш chat_id
    
    message = (
        f"📝 Новий відгук про тест:\n"
        f"Архетип: {archetype} ({similarity_percent}%)\n"
        f"Відгук: {feedback_text}"
    )
    
    try:
        requests.post(
            f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage",
            json={"chat_id": CHAT_ID, "text": message}
        )
    except Exception as e:
        print(f"Помилка відправки: {str(e)}")


def show_results():
    """Відображає результати тесту"""
    clear_output()
    final_scores = calculate_scores(user_answers, questions)
    archetype, similarity_percent, description = determine_archetype(final_scores)
    

    
    # Результати
    result_html = f"""
    <div style="background:#181a1b; color:#e0e0e0; border-radius:16px; padding:40px; max-width:800px; margin:30px auto;">
        <h2 style="color:#4f8cff; text-align:center; margin-bottom:30px;">🎯 Ваш психологічний архетип</h2>
        
        <div style="background:#23272b; border-radius:12px; padding:25px; margin-bottom:25px; border-left:5px solid #4f8cff;">
            <h3 style="color:#4f8cff; margin-bottom:15px;">{archetype}</h3>
            <p style="font-size:1.1em; line-height:1.6;">{description}</p>
        </div>
        
        <div style="background:#23272b; border-radius:12px; padding:20px;">
            <h4 style="color:#4f8cff; margin-bottom:15px;">Ваші бали по категоріях:</h4>
            <div style="display:grid; grid-template-columns:repeat(auto-fit, minmax(150px, 1fr)); gap:15px;">
                {''.join([f'<div style="text-align:center; padding:10px; background:#181a1b; border-radius:8px;"><strong>{cat}</strong><br><span style="color:#4f8cff; font-size:1.2em;">{score}</span></div>' for cat, score in final_scores.items()])}
            </div>
        </div>
        
        <div style="text-align:center; margin-top:25px;">
            <button onclick="location.reload()" style="background:#4f8cff; color:#fff; border:none; padding:12px 24px; border-radius:8px; font-size:1.1em; cursor:pointer;">
                Пройти тест знову
            </button>
        </div>
    </div>
    """
    
    display(HTML(result_html))

    # === Віджет для зворотного зв’язку ===
    feedback_label = widgets.HTML("<b>Залиште ваш відгук про тест:</b>")
    feedback_text = widgets.Textarea(
        placeholder='Ваші враження, побажання, чи співпав результат?',
        layout=widgets.Layout(width='100%', height='80px')
    )
    send_btn = widgets.Button(
        description='Надіслати в Telegram',
        button_style='info',
        layout=widgets.Layout(width='220px', margin='15px 0 0 0')
    )
    feedback_output = widgets.Output()

    def on_send_btn_click(b):
        with feedback_output:
            clear_output()
            if feedback_text.value.strip():
                send_feedback_to_telegram(feedback_text.value, archetype, similarity_percent)
                display(HTML("<span style='color:#4f8cff;'>Відгук надіслано, дякуємо!</span>"))
            else:
                display(HTML("<span style='color:#ff6b6b;'>Введіть текст відгуку!</span>"))

    send_btn.on_click(on_send_btn_click)
    display(widgets.VBox([feedback_label, feedback_text, send_btn, feedback_output]))
    
    
# Запуск тесту
print("🧠 Психологічний тест для фінансових ринків")
print("=" * 50)
show_question(0)


IndentationError: unexpected indent (1297503636.py, line 96)